In [1]:
# Imports
import numpy as np
import pandas as pd
import time
import datetime
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

/home/abhay/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Data Preprocessing

In [2]:
# Defines
YEAR = 2017
THRESHOLD = .60
FILL_NA = 0
DATE_CONVERSION = 'timestamps'
PREPROCESSING = 'MinMax'
KFOLD_SPLITS = 10

In [3]:
# Functions
def check_na(train):
    # Finds the number of missing values in each column
    num_of_na = [train.loc[:,col].isnull().sum() for col in train]
    # Divide by rows for proportion 
    prop_na = [num / train.shape[0] for num in num_of_na]
    # Put the proporitons and column names into a df and sort
    na_df = pd.DataFrame({'prop_na' : prop_na, 'column' : train.columns}).sort_values('prop_na')
    return na_df

In [4]:
# Read csvs
if YEAR == 2016:
    properties = pd.read_csv('properties_2016.csv', low_memory = False)
    train = pd.read_csv('train_2016_v2.csv', low_memory = False)
elif YEAR == 2017:
    properties = pd.read_csv('properties_2017.csv', low_memory = False)
    train = pd.read_csv('train_2017.csv', low_memory = False)
# train has Y and properties has features
# Find row intersection of train and properties
train = train.merge(properties, on = 'parcelid', how = 'left')

In [5]:
print('BEFORE')
print(check_na(train))
# Remove all columns above the THRESHOLD
train = train.loc[:, (train.isnull().sum(axis=0) <= (train.shape[0]*THRESHOLD))]
print('AFTER')
print(check_na(train))

BEFORE
     prop_na                        column
0   0.000000                      parcelid
1   0.000000                      logerror
2   0.000000               transactiondate
27  0.000438                     longitude
34  0.000438     propertycountylandusecode
35  0.000438         propertylandusetypeid
37  0.000438        rawcensustractandblock
39  0.000438                regionidcounty
42  0.000438                       roomcnt
19  0.000438                          fips
7   0.000438                    bedroomcnt
6   0.000438                   bathroomcnt
54  0.000438                assessmentyear
26  0.000438                      latitude
53  0.000451             taxvaluedollarcnt
55  0.000464         landtaxvaluedollarcnt
56  0.000502                     taxamount
41  0.001082                   regionidzip
52  0.001920    structuretaxvaluedollarcnt
13  0.003028  calculatedfinishedsquarefeet
59  0.003621           censustractandblock
49  0.003917                     yearbuilt
21  

In [6]:
# Convert transactiondate strings into floats
date_strings = (train.values[:,2])
date_converted = []
if DATE_CONVERSION == 'timestamps':
    for string in date_strings:
        date_converted.append(time.mktime(datetime.datetime.strptime(string, "%Y-%m-%d").timetuple()))
train['transactiondate'] = np.asarray(date_converted)
# Drop the columns with string and int
train = train.drop(columns=['propertycountylandusecode', 'propertyzoningdesc'])

# Implementation of K-Fold Cross-Validation

In [35]:
# Preprocessing
y = train.values[:,1]
y = y.reshape(y.shape[0],1)
x = train.values[:,2:]
# KFolds
train_index_array = []
test_index_array = []
kf = KFold(n_splits = KFOLD_SPLITS, shuffle = True, random_state = 1)
for train_index, test_index in kf.split(x):
    print("TRAIN:", train_index, "TEST:", test_index)
    train_index_array.append(train_index)
    test_index_array.append(test_index)
MY_INDEX = 6
x_train, x_test = x[train_index_array[MY_INDEX]], x[test_index_array[MY_INDEX]]
y_train, y_test = y[train_index_array[MY_INDEX]], y[test_index_array[MY_INDEX]]

TRAIN: [    0     2     3 ... 77609 77611 77612] TEST: [    1    28    30 ... 77602 77607 77610]
TRAIN: [    0     1     2 ... 77610 77611 77612] TEST: [   10    24    25 ... 77586 77593 77595]
TRAIN: [    0     1     2 ... 77610 77611 77612] TEST: [    3    12    13 ... 77582 77603 77608]
TRAIN: [    0     1     2 ... 77610 77611 77612] TEST: [    8    23    40 ... 77567 77597 77599]
TRAIN: [    0     1     3 ... 77610 77611 77612] TEST: [    2    11    22 ... 77569 77574 77605]
TRAIN: [    0     1     2 ... 77609 77610 77611] TEST: [   33    37    39 ... 77585 77601 77612]
TRAIN: [    0     1     2 ... 77610 77611 77612] TEST: [   26    35    36 ... 77587 77588 77591]
TRAIN: [    1     2     3 ... 77609 77610 77612] TEST: [    0     4     6 ... 77600 77606 77611]
TRAIN: [    0     1     2 ... 77610 77611 77612] TEST: [    5    41    49 ... 77592 77604 77609]
TRAIN: [    0     1     2 ... 77610 77611 77612] TEST: [    9    15    18 ... 77589 77594 77596]


# Implementation of XGBoost Model
##### For k-folds, set MY_INDEX variable as the desired fold index in the above cell and re-run the neural network
##### Execution of only one fold is shown

In [36]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

model = XGBRegressor(max_depth=6, n_estimators=500, learning_rate=0.01)
# reg_cv = GridSearchCV(model, {'max_depth': [4,6,8], 'n_estimators': [500,1000], 
#     'learning_rate': [0.1, 0.01, 0.2, 0.3]}, verbose=1, cv=3)
# reg_cv.fit(x_train, y_train)

In [37]:
# model = XGBRegressor(**reg_cv.best_params_)
model = XGBRegressor(max_depth=4, n_estimators=500, learning_rate=0.01)
model.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=500,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [38]:
prediction = model.predict(x_test)   #predict values
prediction

array([0.04518446, 0.01531258, 0.02346951, ..., 0.01628304, 0.08677837,
       0.03979209], dtype=float32)

In [39]:
from sklearn.metrics import mean_squared_error as mse
mse(y_test, prediction)

0.026139890779744858